In [7]:
import requests

shots_url = "http://stats.nba.com/stats/playergamelog?DateFrom=&DateTo=&LeagueID=00&PlayerID=202391&Season=2016-17&SeasonType=Regular+Season"

# request the URL and parse the JSON
response = requests.get(shots_url)
response.raise_for_status() # raise exception if invalid response
response_json = response.json()
shots = response_json['resultSets'][0]['rowSet']
result = response_json['resultSets'][0]['headers']

# do whatever we want with the shots data
print shots
print "headers", result


 [[u'22016', 202391, u'0021600057', u'NOV 02, 2016', u'BKN vs. DET', u'W', 16, 4, 6, 0.667, 0, 1, 0.0, 2, 2, 1.0, 0, 2, 2, 4, 0, 0, 1, 0, 10, 8, 1], [u'22016', 202391, u'0021600043', u'OCT 31, 2016', u'BKN vs. CHI', u'L', 24, 6, 10, 0.6, 1, 2, 0.5, 1, 2, 0.5, 0, 2, 2, 4, 4, 0, 3, 0, 14, -15, 1], [u'22016', 202391, u'0021600031', u'OCT 29, 2016', u'BKN @ MIL', u'L', 33, 4, 16, 0.25, 1, 7, 0.143, 3, 5, 0.6, 1, 2, 3, 11, 3, 1, 4, 0, 12, 12, 2], [u'22016', 202391, u'0021600019', u'OCT 28, 2016', u'BKN vs. IND', u'W', 34, 6, 12, 0.5, 2, 4, 0.5, 7, 8, 0.875, 0, 9, 9, 9, 0, 1, 2, 1, 21, 0, 2], [u'22016', 202391, u'0021600006', u'OCT 26, 2016', u'BKN @ BOS', u'L', 28, 6, 14, 0.429, 2, 5, 0.4, 4, 5, 0.8, 1, 2, 3, 3, 0, 1, 2, 1, 18, -3, 2]]
headers [u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL', u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS', u'PLUS_MINUS